In [1]:
from DA_yolov8 import *

In [2]:
import sys
print(sys.path)

sys.path.append('/hy-tmp/yolo_soybean/ultralytics')

['/hy-tmp/yolo_soybean/ultralytics', '/usr/local/miniconda3/lib/python38.zip', '/usr/local/miniconda3/lib/python3.8', '/usr/local/miniconda3/lib/python3.8/lib-dynload', '', '/usr/local/miniconda3/lib/python3.8/site-packages']


## Train

In [3]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8m.yaml")  # build a new model from YAML
model = YOLO("/hy-tmp/yolo_soybean/ckpts/yolov8m.pt")  # load a pretrained model (recommended for training)
# Train the model
custom_trainer = partial(CustomTrainer, target_domain_data_cfg="/hy-tmp/yolo_soybean/datasets/DA_dataset/outdoor_domain.yaml")

In [1]:
model.train(custom_trainer,data="/hy-tmp/yolo_soybean/datasets/DA_dataset/all_domain.yaml", name='train_soy',patience=350, epochs=350, imgsz=640, batch=4)

# Evaluation and inference

### Predict seeds

In [2]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8m.yaml")  # build a new model from YAML
model = YOLO("/hy-tmp/yolo_soybean/ckpts/yolo_da.pt")  # load a pretrained model (recommended for training)

In [3]:
def load_labels(label_path):
    """Load label file and count the number of beans based on class ID."""
    count_map = {0: 1, 1: 2, 2: 3, 3: 4}  
    total_count = 0
    with open(label_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            class_id = int(line.split()[0])
            total_count += count_map[class_id]
    return total_count


def count_beans(predictions):
    """Count the number of beans in predictions based on class ID."""
    count_map = {0: 1, 1: 2, 2: 3, 3: 4}  
    total_count = 0
    for prediction in predictions:
        class_id = int(prediction[5])  # column 6:ID
        total_count += count_map[class_id]
    return total_count

In [ ]:
import glob
import os
import numpy as np

test_images_path = '/hy-tmp/yolo_soybean/datasets/DA_dataset/eval/images'
labels_path = '/hy-tmp/yolo_soybean/datasets/DA_dataset/eval/labels'

image_paths = glob.glob(os.path.join(test_images_path, '*.*'))

errors = []
results = []

for image_path in image_paths:
    image_name = os.path.basename(image_path)
    label_name = os.path.splitext(image_name)[0] + '.txt'
    label_path = os.path.join(labels_path, label_name)

    if not os.path.exists(label_path):
        print(f'Label file not found for image {image_name}, skipping...')
        continue

    true_count = load_labels(label_path)
    model_results = model(image_path)

    if model_results and len(model_results) > 0:
        predictions = model_results[0].boxes.data  
        pred_count = count_beans(predictions.cpu().numpy())  
    else:
        pred_count = 0

    error = abs(pred_count - true_count)
    errors.append(error)
    results.append((image_name, true_count, pred_count, error))

# Sort results by error in descending order
results.sort(key=lambda x: x[3], reverse=True)

# Write results to a file
with open('sorted_errors.txt', 'w') as file:
    for result in results:
        file.write(f'Image: {result[0]}, True Count: {result[1]}, Predicted Count: {result[2]}, Error: {result[3]}\n')

mae = np.mean(errors)
print(f'Mean Absolute Error (MAE): {mae}')


### Predict pods

In [ ]:
def load_labels(label_path):
    """Load label file and count the number of beans (detection boxes)."""
    with open(label_path, 'r') as f:
        lines = f.readlines()
    return len(lines)

def count_beans(predictions):
    """Count the number of beans (detection boxes) in predictions."""
    return len(predictions)

import glob
import os
import numpy as np

test_images_path = '/hy-tmp/yolo_soybean/datasets/DA_dataset/eval/images'
labels_path = '/hy-tmp/yolo_soybean/datasets/DA_dataset/eval/labels'

image_paths = glob.glob(os.path.join(test_images_path, '*.*'))

errors = []
results = []

for image_path in image_paths:
    image_name = os.path.basename(image_path)
    label_name = os.path.splitext(image_name)[0] + '.txt'
    label_path = os.path.join(labels_path, label_name)

    if not os.path.exists(label_path):
        print(f'Label file not found for image {image_name}, skipping...')
        continue

    true_count = load_labels(label_path)
    model_results = model(image_path)

    if model_results and len(model_results) > 0:
        predictions = model_results[0].boxes.data  
        pred_count = count_beans(predictions.cpu().numpy())  
    else:
        pred_count = 0

    error = abs(pred_count - true_count)
    errors.append(error)
    results.append((image_name, true_count, pred_count, error))

# Sort results by error in descending order
results.sort(key=lambda x: x[3], reverse=True)

# Write results to a file
with open('sorted_errors_pods.txt', 'w') as file:
    for result in results:
        file.write(f'Image: {result[0]}, True Count: {result[1]}, Predicted Count: {result[2]}, Error: {result[3]}\n')

mae = np.mean(errors)
print(f'Mean Absolute Error (MAE): {mae}')